![logo](../../_static/images/NCI_logo.png)

-------

# Use cdo to do compare model and observation data


In this notebook:

- Load CDO module
- Look at the information of data files
- Concatenate data files
- Data remapping
- Compare between model and obs data 

The following material uses Coupled Model Intercomparison Project (CMIP6) collections. The CMIP6 terms of use are found [here](https://pcmdi.llnl.gov/CMIP6/TermsOfUse/TermsOfUse6-1.html). For more information on the collection, please [click here](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6600_2266_8675_3563).


CDO stands for "Cliamte Data Operators". CDO is a collection of command line Operators to manipulate and analyse Climate and NWP model Data.
Supported data formats are GRIB 1/2, netCDF 3/4, SERVICE, EXTRA and IEG. There are more than 600 operators available. See [here](https://code.mpimet.mpg.de/projects/cdo) for more information about this library.

### Load CDO module

```
$ module load cdo
```

### Let's look at the near surface temperature from the 20th century all-forcing historical simulation based on CESM2.0 model of NCAR 

```
$ ls /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
```

### Have a look at the data file using cdo info

Basic usage:
```
$ cdo info <filename> | less
```

**less** display only one page at a time in the termial. You can move forwards and backwards to see more. Press **q** to quit the view.

Use **ncview** to show the data:
```
$ ncview /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
```

![1](images/cdo_anomoly_nino3.png)

### Let's see which year this file includes

We use function **showyear** to display all the years in this file

Basic usage:
```
$ cdo showyear <file>
```

![2](images/cdo_anomoly_nino2.png)

### Select only 10 year data from the original model file

**basic usage**:

cdo selyear, stratyear/endyear <input.nc> <output.nc>

```
$ cdo selyear,1991/2000 /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc  tas_Amon_CESM2_historical_r1i1p1f1_gn_199101-200012.nc 
```

### HadCRU temperature observation file


```
$ ls -l /short/public/jbw900/HADCRU/HADCRU | grep _tas_
```
I used 'grep _tas_' here to only show tas data

![3](images/cdo_comp1.png)

### Concatenate yearly HADCRU data together

```
$ for year in ` seq 1991 2000 `
$ do 
$ cdo cat /short/public/jbw900/HADCRU/HADCRU/HADCRU_tas_${year}.nc HADCRU_tas_1991_2000.nc
$ done
```
In the loop from year 1991 to 2000, use 'cdo cat <input.nc> <output.nc>' to concatenate all years together.

![4](images/cdo_comp2.png)

### Now we want to see the difference between the model data and observation data

```
$ cdo sub tas_Amon_CESM2_historical_r1i1p1f1_gn_199101-200012.nc HADCRU_tas_1991_2000.nc CESM2_HADCRU_dif.nc
```
cdo sub <input1.nc>  <input2.nc> <output.nc> ---substract input2.nc from input1.nc with the results being output.nc

But got error info as below:

![5](images/cdo_comp3.png)

This is because the resolution of the model data is different from that of the observation data. CDO provides several ways of data interpolation, one of which being 'cdo remapcon'.

**basic usage**
```
cdo remapcon, <input1.nc> <input2.nc> <output.nc>
```
Here input1.nc is the file that we want the resolution to be consistent with.
So, let's do remapping first and then subtraction.

```
$ cdo sub -remapcon,HADCRU_tas_1991_2000.nc tas_Amon_CESM2_historical_r1i1p1f1_gn_199101-200012.nc HADCRU_tas_1991_2000.nc CESM2_HADCRU_dif.nc
```


### Let's see how this model data is different from observation data

```
$ ncview CESM2_HADCRU_dif.nc
```

![5](images/cdo_comp4.png)

![6](images/cdo_anomoly_nino4.png)